# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.


### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [6]:
#just getting a look at the variables
barry= pd.Series(data.columns)
pd.DataFrame(barry.reshape(13,5))

/Users/apple/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app


,0,1,2,3,4
0,id,ad,education,ofjobs,yearsexp
1,honors,volunteer,military,empholes,occupspecific
2,occupbroad,workinschool,email,computerskills,specialskills
3,firstname,sex,race,h,l
4,call,city,kind,adid,fracblack
5,fracwhite,lmedhhinc,fracdropout,fraccolp,linc
6,col,expminreq,schoolreq,eoe,parent_sales
7,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip
8,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip
9,manager,supervisor,secretary,offsupport,salesrep


In [7]:
# group the data by race and call-back, and get the counts for each.
values = pd.DataFrame(data.groupby(['race','call']).size())
values = values.reset_index()
values = values.rename(columns = {0:'count'})
values

,race,call,count
0,b,0.0,2278
1,b,1.0,157
2,w,0.0,2200
3,w,1.0,235


# 1.) What test is appropriate for this problem? Does CLT apply?

## Comparing two independent proportions.  Yes.

# 2. What are the null and alternate hypotheses?

# Ho: p1 = p2
# Ha: p1 != p2

# 3. Compute margin of error, confidence interval, and p-value.


In [8]:
int(values['count'][(values.race == 'w') & (values.call == 1)]) / int(values['count'][(values.race == 'w')].sum())

0.09650924024640657

In [9]:
# pooled estimate of p

p1 = int(values['count'][(values.race == 'w') & (values.call == 1)]) / int(values['count'][(values.race == 'w')].sum())
n1 = int(values['count'][(values.race == 'w')].sum())
p2 = int(values['count'][(values.race == 'b') & (values.call == 1)]) / int(values['count'][(values.race == 'b')].sum())
n2 = int(values['count'][(values.race == 'b')].sum())

sp = ((p1 * n1) + (p2 * n2)) / (n1 + n2)
sp

0.08049281314168377

In [10]:
z_sc = 1.96
ss = 4870

margin = z_sc * np.sqrt((sp * (1-sp)) / (ss))
bottom = sp - z_sc * np.sqrt((sp * (1-sp)) / (ss))
top = sp + z_sc * np.sqrt((sp * (1-sp)) / (ss))

print('margin: ' + str(margin))
print(' ')
print('95% CI: (' + str(bottom) + ' , ' + str(top) + ')')

margin: 0.00764095615545
 
95% CI: (0.0728518569862 , 0.0881337692971)


In [11]:
st_er = np.sqrt(sp*(1-sp)*((1 / n1) + (1 / n2)))
test_stat = (p1 - p2) / st_er
test_stat

4.1084121524343464

# 4. Write a story describing the statistical significance in the context or the original problem.

## We are over 99% confident that...
## The proportion of people with white sounding names who received a call back from employers IS NOT EQUAL to the proportion of people with black sounding names who received a call back

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

## Not Necessarily. We would have to look at the other variables, and make sure there is no correlation between variables.